In [13]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import mysql.connector
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,MultiPolygon,LineString,MultiLineString,LinearRing,Point
from shapely import symmetric_difference
from shapely.ops import nearest_points
from shapely.wkt import loads
import scipy
import time
import numpy as np
import random
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import osmnx as ox
import glob
from itertools import combinations,product
from copy import deepcopy
import os
from lib_contour_lines import *
import mysql.connector


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:

cnx = mysql.connector.connect(user='spirz', 
                              password='this_is_my_PASSWORD_m8',
                              host='localhost',
                              database='dem_from_contours_lines',
                              autocommit=True)
cursor = cnx.cursor(buffered=True,dictionary=True)
contours_lines_table_name='contours_lines'
###
tree_edges_table_name="tree_edges"
####
osm_nodes_table_name="osm_nodes"
osm_edges_table_name="osm_edges"
intersections_table_name="intersections"
2

2

In [15]:
DELETE_TABLES=True
if DELETE_TABLES:
    cmd="""DROP TABLE IF EXISTS `%s`"""%intersections_table_name
    cursor.execute(cmd)
    cmd="""DROP TABLE IF EXISTS `%s`"""%osm_edges_table_name
    cursor.execute(cmd)
    cmd="""DROP TABLE IF EXISTS `%s`"""%osm_nodes_table_name
    cursor.execute(cmd)

2

2

In [16]:
CREATE_TABLES=True
if CREATE_TABLES:
    cmd="""CREATE TABLE `%s`(
    `osm_id` BIGINT NOT NULL,
    `geometry` POINT NOT NULL,
    `elevation` FLOAT,

    PRIMARY KEY (osm_id)
    );
        """%osm_nodes_table_name
    
    cursor.execute(cmd)

    cmd="""CREATE TABLE `%s`(
    `osm_begin` BIGINT NOT NULL,
    `osm_end` BIGINT NOT NULL,
    `osm_key` INT NOT NULL,
    `length` FLOAT NOT NULL,
    `intersects` BOOLEAN,


    FOREIGN KEY (osm_begin) REFERENCES %s(osm_id),
    FOREIGN KEY (osm_end) REFERENCES %s(osm_id),
    PRIMARY KEY (osm_begin,osm_end,osm_key)

    );
        """%(osm_edges_table_name,osm_nodes_table_name,osm_nodes_table_name)
    
    cursor.execute(cmd)

    
    cmd="""CREATE TABLE `%s`(
        `id` INT NOT NULL AUTO_INCREMENT,
        `osm_begin` BIGINT NOT NULL,
        `osm_end` BIGINT NOT NULL,
        `osm_key` INT NOT NULL,
        `edge_coordinate` FLOAT NOT NULL,
        `geometry` POINT NOT NULL,
        `elevation` FLOAT NOT NULL,

        FOREIGN KEY (osm_begin,osm_end,osm_key) REFERENCES %s(osm_begin,osm_end,osm_key),
        UNIQUE (osm_begin,osm_end,osm_key,edge_coordinate),
        PRIMARY KEY (id)

          )
          ENGINE=InnoDB;
        """%(intersections_table_name,osm_edges_table_name)
    cursor.execute(cmd)
    

2

2

In [17]:
t1=time.time()
place_name='Corse, France'
G_osm=ox.graph_from_place(place_name,network_type='drive')
loops=[edge for edge in G_osm.edges() if edge[0]==edge[1]]
G_osm.remove_edges_from(loops)
G_osm=to_multi_graph(G_osm)
osm_crs=ox.graph_to_gdfs(G_osm,edges=False).estimate_utm_crs()
G_osm=ox.project_graph(G_osm,to_crs=osm_crs)
add_missing_geometries(G_osm)
t2=time.time()
print(t2-t1)
len(G_osm.nodes()),len(G_osm.edges())

71.47761225700378


(17435, 20543)

In [18]:
t1=time.time()
osm_edges_df=ox.graph_to_gdfs(G_osm,nodes=False,edges=True)
osm_edges_df['edge']=osm_edges_df.index
osm_edges_df=osm_edges_df.loc[:,['edge','geometry']]
osm_edges_df['number_of_points']=osm_edges_df['geometry'].apply(lambda ls:len(ls.coords))
osm_edges_df=osm_edges_df.set_index(np.arange(len(osm_edges_df)))
osm_edges_df['id_edge']=osm_edges_df.index
t2=time.time()
print(t2-t1)
osm_edges_df

0.37842583656311035


,edge,geometry,number_of_points,id_edge
0,"(14468319, 15905397, 0)","LINESTRING (525532.710 4599879.701, 525543.715...",82,0
1,"(14468319, 1434053206, 0)","LINESTRING (525532.710 4599879.701, 525519.651...",4,1
2,"(14468319, 1434053161, 0)","LINESTRING (525532.710 4599879.701, 525532.752...",20,2
3,"(15905397, 1976904946, 0)","LINESTRING (526252.098 4600660.017, 526286.419...",12,3
4,"(15905397, 1976906939, 0)","LINESTRING (526252.098 4600660.017, 526253.327...",7,4
...,...,...,...,...
20538,"(9744995304, 9744995311, 0)","LINESTRING (496541.756 4720170.475, 496521.473...",2,20538
20539,"(9744995304, 9744995310, 0)","LINESTRING (496541.756 4720170.475, 496558.928...",7,20539
20540,"(9964974606, 9964974609, 0)","LINESTRING (494174.708 4719764.250, 494163.668...",4,20540
20541,"(9964974610, 9964974611, 0)","LINESTRING (494100.379 4719512.527, 494038.489...",2,20541


In [19]:
t1=time.time()
intersection=increasing_elevations_intersections(osm_edges_df,osm_crs,cursor)
t2=time.time()
print('total : %f'%(t2-t1))

0.000000<=elevation<50.000000 :28.626975 
50.000000<=elevation<100.000000 :25.879143 
100.000000<=elevation<150.000000 :17.958982 
150.000000<=elevation<200.000000 :16.332238 
200.000000<=elevation<250.000000 :14.629966 
250.000000<=elevation<300.000000 :14.004121 
300.000000<=elevation<350.000000 :13.782105 
350.000000<=elevation<400.000000 :13.621201 
400.000000<=elevation<450.000000 :13.349376 
450.000000<=elevation<500.000000 :14.026862 
500.000000<=elevation<550.000000 :12.181399 
550.000000<=elevation<600.000000 :9.424916 
600.000000<=elevation<650.000000 :8.676356 
650.000000<=elevation<700.000000 :8.202727 
700.000000<=elevation<750.000000 :7.396882 
750.000000<=elevation<800.000000 :6.964261 
800.000000<=elevation<850.000000 :6.619977 
850.000000<=elevation<900.000000 :5.901978 
900.000000<=elevation<950.000000 :5.095436 
950.000000<=elevation<1000.000000 :4.601361 
1000.000000<=elevation<1050.000000 :4.421775 
1050.000000<=elevation<1100.000000 :3.921535 
1100.000000<=elevati

In [20]:
len(intersection)

84783

In [21]:
84783

84783

In [22]:
162992

162992

In [23]:
ADD_OSM_DATA=True
if ADD_OSM_DATA:
    t1=time.time()
    edges=osm_edges_df['edge']
    osm_nodes_data=nx.edge_subgraph(G_osm,edges).nodes(data=True)
    
    string_list=',\n'.join(['(%i,ST_GeomFromText(\'%s\'))'%(node,Point(data['lon'],data['lat']).wkt) for node,data in osm_nodes_data])
    cmd="INSERT IGNORE INTO %s(osm_id,geometry) VALUES %s;"%(osm_nodes_table_name,string_list)
    cursor.execute(cmd)

    intersecting_edges=intersection['edge'].unique()
    string_list=',\n'.join(['(%i,%i,%i,%f,TRUE)'%(edge[0],edge[1],edge[2],G_osm.get_edge_data(*edge)['length']) for edge in intersecting_edges])
    cmd="INSERT INTO %s(osm_begin,osm_end,osm_key,length,intersects) VALUES %s;"%(osm_edges_table_name,string_list)
    cursor.execute(cmd)

    inside_edges=set(edges).difference(intersecting_edges)

    string_list=',\n'.join(['(%i,%i,%i,%f,False)'%(edge[0],edge[1],edge[2],G_osm.get_edge_data(*edge)['length']) for edge in inside_edges])
    cmd="INSERT INTO %s(osm_begin,osm_end,osm_key,length,intersects) VALUES %s;"%(osm_edges_table_name,string_list)
    cursor.execute(cmd)


    ###


    ###SOLVE IGNORE




    ###
    intersection=intersection.to_crs('epsg:4326')
#    step=10000
#    for k in range(0,len(intersection)):
#        sub_intersection=intersecion.iloc[k:min(len(intersection,k+step)]
    string_list=',\n'.join(['(%i,%i,%i,%f,%f,ST_GeomFromText(\'%s\'))'%(row['edge'][0],row['edge'][1],row['edge'][2],row['edge_coordinate'],row['elevation'],row['geometry'].wkt)  for _,row in intersection.iterrows()])
    cmd="INSERT INTO %s (`osm_begin`,`osm_end`,`osm_key`,`edge_coordinate`,`elevation`,`geometry`) VALUES %s;"%(intersections_table_name,string_list)
    cursor.execute(cmd)
    
    t2=time.time()
    print(t2-t1)
2

11.875648021697998


2

In [12]:
len(intersection)

84783